First we need to import the packages and functions needed for this analysis

In [1]:
#####################################
### import the used functions
#####################################
#for umap you need the umap-learn package
import umap.umap_ as umap
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import PandasTools
import pandas as pd
import catboost
import shap
import numpy as np

After this we will define several functions needed for our analysis.

In [2]:

#######################################
###define the used functions
#######################################

#define a function that trnasforms smiles RDKit Mol objects
def smiles_to_mols(smis):
    """Convert a list of smiles to a list of RDKit Mol objects"""
    if not isinstance(smis, list):
        raise TypeError("Please provide smiles as a list")
    mols = []
    successful_inds = []
    for ind, smi in enumerate(smis):
        # RDKit will sometimes 'fail' but instead of throwing an
        # error it will (sometimes!) print a warning and return None.
        m = Chem.MolFromSmiles(smi, sanitize=True)
        if m is None:
            print("Mol generation failed for", smi, "(index", ind, ")\n")
        else:
            mols.append(m)
            successful_inds.append(ind)
    return mols, successful_inds


# define a function to calculate count based morgan fingerprints
def calc_morgan_counts_for_smiles(mol, radius=2, nBits=2048):
    try:
        fpCounts = AllChem.GetHashedMorganFingerprint(
            mol, radius, nBits=nBits, includeRedundantEnvironments=False)
        array = np.zeros((0,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fpCounts, array)
    except:
        print("Failure in calc_morgan_counts_for_smiles()")
        array = np.repeat(np.nan, nBits)

    return array


#get the substructure indices for a specfied radius around an atomID
def getSubstructIndex(mol,atomID,radius):
    if radius>0:
        env = Chem.FindAtomEnvironmentOfRadiusN(mol,radius,atomID)
        atomsToUse=[]
        for b in env:
            atomsToUse.append(mol.GetBondWithIdx(b).GetBeginAtomIdx())
            atomsToUse.append(mol.GetBondWithIdx(b).GetEndAtomIdx())
        atomsToUse = list(set(atomsToUse))
    else:
        atomsToUse = [atomID]
        env=None
    return(atomsToUse)

#get the substructure indices for each fingerprint bit
def calc_indices_for_fingerprints(smiles,radius = 2,nBits=2048):
   mol = Chem.MolFromSmiles(smiles,sanitize = True)
   bi = {}
   fp = AllChem.GetHashedMorganFingerprint(mol,radius = radius, nBits=nBits, includeRedundantEnvironments=False,bitInfo=bi)
   fp = fp.GetNonzeroElements()
   #weights = [0  for i in mol.GetAtoms()]
   #weights[slice(mol.GetSubstructMatch(Chem.MolFromSmiles('CNC(=O)C(C)(C)C',sanitize = True)))] += 1


   fp_indices = {}
   for fragment_id, count in fp.items():
      sub_indices = []
      for i in range(count):
         #print(i)
         root, radius = bi[fragment_id][i]
         subindex = getSubstructIndex(mol,root,radius)
         sub_indices = sub_indices + subindex

      sub_indices = list(set(sub_indices))

      fp_indices[fragment_id] = sub_indices
   fp_indices = {"fp_morgan_counts_" +  str(k): v for k, v in fp_indices.items() }

   return(fp_indices)

#we also need a function that calculates the attribution of each atom to the final 
#prediction, for direction you can choose between "up": all positive attributions
#"down": all negative attributions and recommended "both": summing up over all attributions
def atom_attributions(mol, indices, shap_vals, featureNames, direction="both"):
    """
    Sum up the attributions to the model predictions (Shap values) for each atom in mol.

    Parameters:
        mol: rdkit molecule object
        indices (dict): a dictionary with keys = feature names and values = list of atom positions in mol (output of calc_indices_for_fingerprints())
        shap_vals: a numpy array of dimensions (len(featureNames))
        featureNames (np.array): numpy array containing the feature names for shap_vals
        direction (str): whether to consider positive (up), negative (down) or both shap_vals contributions

    Returns:
        np.array: each element = 1 atom of mol and the values are the summed contributions from shap_vals
    """
    ## TO DO: add support for MACCS keys

    # get for each fingerprint bit the atom numbers mapping to that bit and initialise weights to 0
    curWeights = np.array([0 for i in range(mol.GetNumAtoms())], 'float64')

    # step through each index (dictionary)
    for j in indices:
        # retrieve index to subset into the featureNames and the corresponding shap values array
        curFPIndex = [i for i, s in enumerate(featureNames) if s == j]
        if len(curFPIndex) == 0:
            continue
        if len(curFPIndex) > 1:
            raise IndexError("Found several matches for feature " + j)
        # do something only if (i) the fingerprint bit was in the selected features
        # and (ii) only if the retrieved shap value is positive/negative (direction argument)
        if (direction == "up" and shap_vals[curFPIndex[0]] > 0) or (direction == "down" and shap_vals[curFPIndex[0]] < 0) or (direction == "both"):
            # use the atom numbers ('positions') to update the curWeights array, where
            # each atom of the current molecule is represented
            # one atom may be part of several fingerprints so the summing has to be done across
            # all fingerprints
            for pos in indices[j]:
                #value = (curWeights[pos] + shap_vals[curFPIndex[0]])
                value = (curWeights[pos] + (shap_vals[curFPIndex[0]]/len(indices[j])))
                curWeights[pos] = value

    return curWeights


To test the code we define some smiles, this is just a proof of concept, please feel free to add smiles of your own to get abigger dataset.

In [3]:

#######################################################
##define some smiles
#######################################################

smiles = ["c1cccc[nH+]1",
      "C[N+](C)(C)C","c1ccccc1[NH3+]",
      "CC(=O)[O-]","c1ccccc1[O-]",
      "CCS",
      "C[N-]S(=O)(=O)C",
      "C[N-]C=C","C[N-]N=C",
      "c1ccc[n-]1",
      "CC[N-]C(=O)CC"]

Before we can use the smiles some preprocessing steps have to be performed to make them canonical and also to store them as mols for use in Rdkit

In [4]:

################################################################################
##make sure that the smiles are canonical to have full compatibility with CIME
################################################################################

#canonicalize smiles
mols, successful_inds = smiles_to_mols(smiles)
# get canonical smiles
smiles = [Chem.MolToSmiles(m) for m in mols]

#generate some names for the smiles
smiles_names = [i for i in range(len(smiles))]

Before we continue we store the smiles also as a data frame

In [5]:
##################################################
# put the smiles and info into a data frame
###################################################
sdfExportData = pd.DataFrame({'names': smiles_names,
        'smiles': smiles})

Now that we did that we can calculate the logp

In [6]:
################################################################################
##calculate the logp for each of them
################################################################################

logp = [Descriptors.MolLogP(i) for i in mols]

The basis for our model are count based morgan fingerprints which we calculate and store in a matrix format

In [7]:

##############################################
##calculate the morgan fingerprints
################################################################################

# calculate the morgan counts fingerprints
featuresMorganCounts = [calc_morgan_counts_for_smiles(
    mol, radius=2, nBits=2048) for mol in mols]
columns = ["fp_morgan_counts_" + str(i) for i in range(2048)]
featuresMorganCounts = pd.DataFrame(featuresMorganCounts, columns=[
                                  "fp_morgan_counts_" + str(i) for i in range(2048)])

Now we can train the model

In [8]:

##################################################
##train a model on the values
##################################################

model = catboost.CatBoostRegressor()
# Fit model
model.fit(featuresMorganCounts, logp)

Learning rate set to 0.020078
0:	learn: 0.6529934	total: 140ms	remaining: 2m 19s
1:	learn: 0.6501014	total: 141ms	remaining: 1m 10s
2:	learn: 0.6468481	total: 141ms	remaining: 47s
3:	learn: 0.6441279	total: 142ms	remaining: 35.3s
4:	learn: 0.6414791	total: 143ms	remaining: 28.4s
5:	learn: 0.6383103	total: 143ms	remaining: 23.7s
6:	learn: 0.6355829	total: 144ms	remaining: 20.4s
7:	learn: 0.6318449	total: 145ms	remaining: 17.9s
8:	learn: 0.6284757	total: 145ms	remaining: 16s
9:	learn: 0.6257633	total: 146ms	remaining: 14.4s
10:	learn: 0.6229713	total: 146ms	remaining: 13.2s
11:	learn: 0.6196277	total: 147ms	remaining: 12.1s
12:	learn: 0.6170333	total: 147ms	remaining: 11.2s
13:	learn: 0.6137229	total: 148ms	remaining: 10.4s
14:	learn: 0.6111082	total: 148ms	remaining: 9.74s
15:	learn: 0.6081114	total: 149ms	remaining: 9.15s
16:	learn: 0.6054127	total: 149ms	remaining: 8.64s
17:	learn: 0.6024950	total: 150ms	remaining: 8.18s
18:	learn: 0.5998816	total: 150ms	remaining: 7.76s
19:	learn: 0.

326:	learn: 0.1450785	total: 311ms	remaining: 640ms
327:	learn: 0.1443717	total: 311ms	remaining: 638ms
328:	learn: 0.1436666	total: 312ms	remaining: 636ms
329:	learn: 0.1429653	total: 313ms	remaining: 635ms
330:	learn: 0.1422676	total: 313ms	remaining: 633ms
331:	learn: 0.1415737	total: 314ms	remaining: 631ms
332:	learn: 0.1408834	total: 314ms	remaining: 630ms
333:	learn: 0.1401967	total: 315ms	remaining: 628ms
334:	learn: 0.1395137	total: 315ms	remaining: 626ms
335:	learn: 0.1388360	total: 316ms	remaining: 624ms
336:	learn: 0.1381601	total: 317ms	remaining: 623ms
337:	learn: 0.1374878	total: 317ms	remaining: 621ms
338:	learn: 0.1368207	total: 318ms	remaining: 619ms
339:	learn: 0.1361555	total: 318ms	remaining: 618ms
340:	learn: 0.1354938	total: 319ms	remaining: 616ms
341:	learn: 0.1348355	total: 319ms	remaining: 614ms
342:	learn: 0.1341808	total: 320ms	remaining: 613ms
343:	learn: 0.1335295	total: 320ms	remaining: 611ms
344:	learn: 0.1328817	total: 321ms	remaining: 610ms
345:	learn: 

567:	learn: 0.0435262	total: 453ms	remaining: 344ms
568:	learn: 0.0433158	total: 454ms	remaining: 344ms
569:	learn: 0.0431066	total: 454ms	remaining: 343ms
570:	learn: 0.0428985	total: 455ms	remaining: 342ms
571:	learn: 0.0426916	total: 455ms	remaining: 341ms
572:	learn: 0.0424858	total: 456ms	remaining: 340ms
573:	learn: 0.0422813	total: 457ms	remaining: 339ms
574:	learn: 0.0420778	total: 457ms	remaining: 338ms
575:	learn: 0.0418755	total: 458ms	remaining: 337ms
576:	learn: 0.0416743	total: 458ms	remaining: 336ms
577:	learn: 0.0414743	total: 459ms	remaining: 335ms
578:	learn: 0.0412754	total: 459ms	remaining: 334ms
579:	learn: 0.0410776	total: 460ms	remaining: 333ms
580:	learn: 0.0408809	total: 461ms	remaining: 332ms
581:	learn: 0.0406584	total: 461ms	remaining: 331ms
582:	learn: 0.0404374	total: 462ms	remaining: 330ms
583:	learn: 0.0402437	total: 462ms	remaining: 329ms
584:	learn: 0.0400249	total: 463ms	remaining: 328ms
585:	learn: 0.0398075	total: 464ms	remaining: 328ms
586:	learn: 

811:	learn: 0.0129712	total: 596ms	remaining: 138ms
812:	learn: 0.0128991	total: 596ms	remaining: 137ms
813:	learn: 0.0128365	total: 597ms	remaining: 136ms
814:	learn: 0.0127743	total: 597ms	remaining: 136ms
815:	learn: 0.0127083	total: 598ms	remaining: 135ms
816:	learn: 0.0126511	total: 599ms	remaining: 134ms
817:	learn: 0.0125808	total: 599ms	remaining: 133ms
818:	learn: 0.0125198	total: 600ms	remaining: 133ms
819:	learn: 0.0124592	total: 600ms	remaining: 132ms
820:	learn: 0.0123948	total: 601ms	remaining: 131ms
821:	learn: 0.0123392	total: 602ms	remaining: 130ms
822:	learn: 0.0122705	total: 602ms	remaining: 129ms
823:	learn: 0.0122111	total: 603ms	remaining: 129ms
824:	learn: 0.0121521	total: 603ms	remaining: 128ms
825:	learn: 0.0120845	total: 604ms	remaining: 127ms
826:	learn: 0.0120312	total: 604ms	remaining: 126ms
827:	learn: 0.0119772	total: 605ms	remaining: 126ms
828:	learn: 0.0119153	total: 606ms	remaining: 125ms
829:	learn: 0.0118619	total: 607ms	remaining: 124ms
830:	learn: 

Now that we trained a model we can gat the shap values for each fingerprint bit.

In [9]:
##################################################
##calculate the shap values
##################################################

explainer = shap.TreeExplainer(model)
shap_vals = explainer.shap_values(featuresMorganCounts)


As we now have the shap values we can use them to calculate a distance matrix based on spearman correlation. Use the distance to for UMAP and store the emebdding vectors in the sdfExportData

In [10]:

#######################################################################
## Calculate umap based on the shap values and add to the data frame ##
#######################################################################

#We use spearman correlation as a proximity measure, scale it to a range
#between 0 and 1 and transform it to a distance to be used in umap
umap_dist = 1 - (pd.DataFrame(shap_vals).T.corr(method="spearman") + 1) / 2

my_umap = umap.UMAP(metric='precomputed')

my_umap_fit = my_umap.fit_transform(umap_dist)

print("Finished UMAP")

#get the coordinates
x = my_umap_fit[:, 0]
y = my_umap_fit[:, 1]

#add the coordinates to the data object
sdfExportData["x"] = x
sdfExportData["y"] = y


using precomputed metric; inverse_transform will be unavailable
n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1


Finished UMAP


Now lets add some dummy groupLabels

In [11]:
groups = np.zeros((len(sdfExportData),))
groups[3:] = 1
sdfExportData["groupLabel"] = groups

Now lets use the shap values to assign atom attributions based on the model for logp prediction

In [12]:
####################################################
#now assign weights based on the model
###################################################

#first we have to calculate the atom indices associated with each fingerprint bit
indices =  [calc_indices_for_fingerprints(smi,radius = 2,nBits=2048) for smi in smiles]

#we also store the column names of the fingerprint matrix
featureNames = featuresMorganCounts.columns

#set direction to "up" if you only want the positive attributions
#to "down" for only the positive attributions and "both" summing up over positive and
#negative attributions
modelAttributions = [list(atom_attributions(mols[i], indices[i], shap_vals[i],featureNames , direction="both")) for i in range(len(smiles))]

####################################################################################
##Add the modelAttributions to the logp as one string for each compound
####################################################################################

sdfExportData["atom.Contribution.rep_modelAttributions"] = [' '.join([str(round(i, 4)) for i in modelAttributions[j]]) for
                                                           j in range(len(modelAttributions))]


To compare the model attribution against them we also calculate the crippen contributions

In [13]:
######################################################
##Calculate the Crippen Contribs to the logp values
######################################################

logp_contribs = [[x for x, y in rdMolDescriptors._CalcCrippenContribs(mols[i])] for i in range(len(mols))]

sdfExportData["atom.Contribution.rep_CrippenContribs"] = [' '.join([str(round(i, 4)) for i in logp_contribs[j]]) for j
                                                          in range(len(logp_contribs))]


As both attributions (Crippen and Model) were stored in the sdfExportData we prepare the data frame fro export to sdf

In [14]:
#######################################################################################
#add the chemical info to the data frame
#######################################################################################
# smilesCol: in which column the smiles are located
PandasTools.AddMoleculeColumnToFrame(sdfExportData, smilesCol='smiles', molCol='ROMol', includeFingerprints=False)
# to avoid duplicate smiles column
del sdfExportData['smiles']


Now we can write the data frame as an sdf file for use in CIME

In [15]:
########################################################################
###Export as an sdf file you can read into CIME
########################################################################

PandasTools.WriteSDF(sdfExportData,"SDF_OUTPUT.sdf",
                     properties=list(sdfExportData.columns), idName="RowID")

